In [36]:
import psycopg2

### Sample postgres table:

```
CREATE TABLE edge_table (
    id BIGSERIAL,
    source BIGINT,
    target BIGINT,
    cost FLOAT,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT
);
```

### Postgres connection

In [37]:
conn = psycopg2.connect(
    host="localhost",
    database="krakow",
    user="postgres",
    password="postgres")

### Data loading

In [38]:
conn.autocommit = True
cursor = conn.cursor() 

In [39]:
drop_table = "DROP TABLE IF EXISTS edge_table CASCADE;"

create_table = """
    CREATE TABLE edge_table (
    source BIGINT,
    target BIGINT,
    osmid BIGINT,
    length FLOAT,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT);
"""

create_extension = """
    CREATE EXTENSION IF NOT EXISTS postgis;
    CREATE EXTENSION IF NOT EXISTS pgrouting;
"""

create_index = "CREATE INDEX index_edge_table ON edge_table (source, target);"

### Drop table (if exists)

In [40]:
cursor.execute(drop_table)

### Create table

In [30]:
cursor.execute(create_table)

### Load data from csv

In [31]:
# TODO: remember to change path to .csv file
csv_file_name = "/home/pcend/Piotr/kod/lisboa-routing/data/krakow_medium/edge_table_sql.csv"
copy_csv_query = "COPY edge_table FROM STDIN DELIMITER ',' CSV HEADER"

In [32]:
cursor.copy_expert(copy_csv_query, open(csv_file_name, "r"))

### Create index

In [33]:
cursor.execute(create_index)

### pgRouting and postgis

In [34]:
cursor.execute(create_extension)

### Search queries

In [12]:
ids = (356926768, 2104495834) # short
ids = (22766069, 1222047203) # medium

In [13]:
dijkstra = f"""
    SELECT * FROM pgr_dijkstra(
    'SELECT osmid as id, source, target, length as cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar = f"""
    SELECT * FROM pgr_astar(
    'SELECT osmid as id, source, target, length as cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

dijkstra_benchmark = f"""
    EXPLAIN ANALYZE 
    SELECT * FROM pgr_dijkstra(
    'SELECT osmid as id, source, target, length as cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar_benchmark = f"""
    EXPLAIN ANALYZE
    SELECT * FROM pgr_astar(
    'SELECT osmid as id, source, target, length as cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

In [35]:
print(astar)


    SELECT * FROM pgr_astar(
    'SELECT osmid as id, source, target, length as cost, x1, y1, x2, y2 FROM edge_table',
    22766069, 1222047203,
    directed => false);



In [19]:
cursor.execute(dijkstra)

In [20]:
result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['seq', 'path_seq', 'start_vid', 'end_vid', 'node', 'edge', 'cost', 'agg_cost']


In [21]:
cursor.execute(astar)

In [22]:
result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['seq', 'path_seq', 'node', 'edge', 'cost', 'agg_cost']


### Disconnect

In [23]:
conn.commit() 
conn.close() 